In [1]:
import numpy as np
import scipy.optimize
import scipy.special
import scipy.stats as st

import bokeh.plotting
import bokeh.io

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
def bounds_solver(F, x1, x2, p1, p2, bounds1=(-np.inf, np.inf), bounds2=(-np.inf, np.inf)):
    def rootfun(a, b):
        if not (bounds1[0] <= a <= bounds1[1]) or not (bounds2[0] <= b <= bounds2[1]):
            return 
        return np.array([p1 - F(x1, a, b), p2 - F(x2, a, b)])

    return scipy.optimize

In [3]:
F = st.beta.cdf
def G(params, F, x1, x2, p1, p2):
    return np.array([p1 - F(x1, *params), p2 - F(x2, *params)])

G(np.array([1, 1]), F, 0.2, 0.7, 0.025, 0.975)

array([-0.175,  0.275])

In [4]:
st.beta.cdf(0.7, 6.015658766792033, 7.596631444667818)

0.9749931856718963

In [5]:
scipy.optimize.root?

Signature:
scipy.optimize.root(
    fun,
    x0,
    args=(),
    method='hybr',
    jac=None,
    tol=None,
    callback=None,
    options=None,
)
Docstring:
Find a root of a vector function.

Parameters
----------
fun : callable
    A vector function to find a root of.
x0 : ndarray
    Initial guess.
args : tuple, optional
    Extra arguments passed to the objective function and its Jacobian.
method : str, optional
    Type of solver. Should be one of

        - 'hybr'             :ref:`(see here) <optimize.root-hybr>`
        - 'lm'               :ref:`(see here) <optimize.root-lm>`
        - 'broyden1'         :ref:`(see here) <optimize.root-broyden1>`
        - 'broyden2'         :ref:`(see here) <optimize.root-broyden2>`
        - 'anderson'         :ref:`(see here) <optimize.root-anderson>`
        - 'linearmixing'     :ref:`(see here) <optimize.root-linearmixing>`
        - 'diagbroyden'      :ref:`(see here) <optimize.root-diagbroyden>`
        - 'excitingmixing'   :ref:`(see 

In [6]:
x1, x2, p1, p2 = 0.3, 0.45, 0.025, 0.975


def root_fun(params, x1, x2, p1, p2):
    F = st.beta.cdf
    return np.array([p1 - F(x1, *params), p2 - F(x2, *params)])


res = scipy.optimize.least_squares(
    root_fun, 
    np.array([1, 1]), 
    args=(x1, x2, p1, p2),
    bounds=([0, 0], [np.inf, np.inf])
)

%timeit scipy.optimize.root(root_fun, np.array([1, 1]), args=(x1, x2, p1, p2), method='hybr')

7.03 ms ± 261 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
import eqtk

In [10]:
np.cbrt(1e-16)

4.6415888336127786e-06

In [35]:
"""
Trust region method of optimization of convex objective function.
"""

import warnings

import numpy as np
import scipy.linalg
import numba
from copy import deepcopy

# Throw RuntimeWarnings as errors
warnings.simplefilter('error', RuntimeWarning)

def jac(x, f, eps=4e-6, args=()):
    n = len(x)
    J = np.empty((n, n))
    x_plus = np.copy(x)
    x_minus = np.copy(x)
    for j in range(n):
        x_plus[j] += eps
        x_minus[j] -= eps
        J[:, j] = (f(x_plus) - f(x_minus)) / 2 / eps
        x_plus[j] -= eps
        x_minus[j] += eps

    return J

In [36]:
def f(x):
    return np.array([x[0], 10 * x[0] / (x[0] + 0.1) + 2 * x[1]**2])
#    return np.array([x[1]**2 - 1, np.sin(x[0]) - x[1]])

x = np.array([2.2, 1.2])

jac(x, f)

array([[1.        , 0.        ],
       [0.18903592, 4.8       ]])

In [37]:
1 / 2.3**2

0.18903591682419663

In [29]:
eps = 4e-6
x_plus = x
x_minus = x
i = 0
x_plus[i] += eps
x_minus[i] -= eps
(f(x_plus) - f(x_minus)) / 2 / eps

array([0., 0.])

In [31]:
x

array([3.2, 1.2])

In [26]:
x_plus[0] += 1
x_plus

array([3.2, 1.2])

In [ ]:
# ####################
def trust_region_convex_unconstrained(x0, f, grad_f, hes_f, params=(),
                                      tol=0.000000001, max_iters=10000,
                                      delta_bar=1000.0, eta=0.125,
                                      min_delta=1e-12, report_steps=False):
    """
    Performs a unconstrained trust region optimization on a convex
    function where the gradient and hessian have analytical forms and
    the functions to calculate them are given.  This algorithm is
    given in Nocedal and Wright, Numerical Optimization, 1999, page
    68, with the dogleg method on page 71.

    x0: initial guess of optimal x
    f: the objective function of the form f(x, *params)
    grad_f: Function to compute the gradient of f.  grad_f(x, *params)
    hes_f: Function to compute the gradient of f.  hes_f(x, *params)

    In the objective function is strictly convex, the trust region
    algorithm is globally convergent.  However, on occasion, when
    close to the optimum, the gradient may be close to zero, but not
    within tolerance, but the value of the ojbective function is the
    optimal value to within machine precision.  This results in the
    criterion for shrinking the trust region to be triggered (rho
    \approx 0), and the trust region shrinks to a infinitum.  As a
    remedy for this, we specify min_delta to be the minimal trust
    region redius allowed.  When the trust region radius gest below
    this value, we are very very close to the minimum.  The algorithm
    then proceeeds with Newton steps until convergence is achieved or
    the Newton step fails to decrease the norm of the gradient.
    """

    # DEBUG
    if report_steps:
        f_out = open('eqcon_iters.txt', 'w')
    # #########

    # Initializations
    delta = 0.99 * delta_bar
    iters = 0
    n_no_step = 0
    x = deepcopy(x0)
    max_grad = tol + 1.0  # Initialized just to get started
    step_tally = np.zeros(6)

    # Calculate the gradient
    g = grad_f(x, *params)

    # Compute Hessian
    B = hes_f(x, *params)

    # Run the trust region
    while iters < max_iters and check_tol(g, tol) and delta >= min_delta:

        # Solve for search direction
        p, search_result = search_direction_dogleg(g, B, delta)
        step_tally[search_result - 1] += 1

        # Calculate rho, ratio of actual to predicted reduction
        rho = compute_rho(x, p, f, g, B, params)

        # Adjust delta
        if rho < 0.25:
            delta /= 4.0
        elif rho > 0.75 and abs(np.linalg.norm(p) - delta) < 1e-12:
            delta = min(2.0 * delta, delta_bar)

        # Make step based on rho
        if rho > eta:
            x += p

            # Compute the new gradient
            g = grad_f(x, *params)

            # Compute new Hessian
            B = hes_f(x, *params)

        # DEBUG
        if report_steps:
            for i in range(len(x)):
                f_out.write('%.18e\t' % x[i])
            for i in range(len(g)):
                f_out.write('%.18e\t' % g[i])
            for i in range(B.shape[0]):
                for j in range(B.shape[1]):
                    f_out.write('%18.6e\t' % B[i][j])
            f_out.write('%.18e\t' % rho)
            f_out.write('%.18e\n' % delta)
        # #########

        iters += 1

    if delta <= min_delta:
        # Try Newton stepping to the solution (only Newton step if norm(g) decr)
        newton_success = True
        while newton_success and check_tol(g, tol) and iters < max_iters:
            try:
                chol_lower = scipy.linalg.cholesky(
                    B, lower=True, overwrite_a=False)
                pB = -scipy.linalg.cho_solve((chol_lower, True), g)
                new_x = x + pB
                new_g = grad_f(new_x, *params)

                # If decreased the gradient, take the step.
                if np.linalg.norm(new_g) < np.linalg.norm(g):
                    x = new_x
                    B = hes_f(x, *params)
                    step_tally[0] += 1
                else:
                    newton_success = False
                iters += 1
            except:
                newton_success = False
        if newton_success and not check_tol(g, tol):
            converged = True
        else:
            converged = False
    else:
        converged = True

    if not converged and iters == max_iters:
        converged = False
    else:
        converged = True

    # DEBUG
    if report_steps:
        f_out.close()
    # #########

    return x, converged, step_tally

# ####################
def search_direction_dogleg(g, B, delta):
    """
    Computes the search direction using the dogleg method (Nocedal and Wright,
    page 71).  Notation is consistent with that in this reference.

    p = direction to step
    g = gradient
    B = Hessian
    delta = radius of trust region

    Due to the construction of the problem, the minimization routine
    to find tau can be solved exactly by solving a quadratic.  There
    can be precision issues with this, so this is checked.  If the
    argument of the square root in the quadratic formula is negative,
    there must be a precision error, as such a situation is not
    possible in the construction of the problem.

    Returns:
      p, the search direction and also an integer,

      1 if step was a pure Newton step (didn't hit trust region boundary)
      2 if the step was purely Cauchy in nature (hit trust region boundary)
      3 if the step was a dogleg step (part Newton and part Cauchy)
      4 if Cholesky decomposition failed and we had to take a Cauchy step
      5 if Cholesky decompostion failed but we would've taken Cauchy step anyway
      6 if the dogleg calculation failed (should never happen)
    """

    # Useful to have around
    delta2 = delta**2

    # Compute Newton step, pB
    try:
        chol_lower = scipy.linalg.cholesky(B, lower=True, overwrite_a=False)
        cholesky_success = True
    except:
        cholesky_success = False

    if cholesky_success:
        pB = -scipy.linalg.cho_solve((chol_lower, True), g)

        # If Newton step is in trust region, take it
        pB2 = np.dot(pB, pB)
        if pB2 <= delta2:
            return pB, 1

    # Compute Cauchy step, pU
    pU = -np.dot(g, g) / np.dot(g, np.dot(B, g)) * g
    pU2 = np.dot(pU, pU)
    if pU2 >= delta2: # In this case, just take Cauchy step, 0 < tau <= 1
        tau = np.sqrt(delta2 / pU2)
        p = tau * pU
        if not cholesky_success:
            return p, 5  # Cholesky failure, but doesn't matter, just use Cauchy
        else:
            return p, 2  # Took Cauchy step

    if not cholesky_success: # Failed computing Newton, have to take Cauchy
        return pU, 4

    # Compute the dogleg step
    pBpU = np.dot(pB, pU)  # Needed for dogleg computation

    # Compute constants for quadratic forumla for solving
    # ||pU + beta (pB-pU)||^2 = delta2, with beta = tau - 1
    a = pB2 + pU2 - 2.0 * pBpU  # a > 0
    b = 2.0 * (pBpU - pU2)       # b can be positive or negative
    c = pU2 - delta2            # c < 0 since pU2 < delta 2 to get here
    q = -0.5 * (b + np.sign(b) * np.sqrt(b**2 - 4.0 * a * c))

    # Choose correct (positive) root (don't have to worry about a = 0 because
    # if pU \approx pB, we would have already taken Newton step
    if abs(b) < 1e-12:
        beta = np.sqrt(-c / a)
    elif b < 0.0:
        beta = q / a
    else:  # b > 0
        beta = c / q

    # Take the dogleg step
    if 0.0 <= beta <= 1.0:  # Should always be the case
        p = pU + beta * (pB - pU)
        return p, 3
    else: # Something is messed up, take Cauchy step (should never get here)
        return pU, 6


# ####################
def compute_rho(x, p, f, g, B, params):
    """
    Calculates rho based on equations 4.4 and 4.1 of Nocedal and
    Wright.  This is the ratio of the actual correction based on the
    stepping a length delta along the search direction to the
    predicted correction of taking the same step.

    Function returns -1 if there is an overflow warning in the
    calculation of rho.
    """

    try:
        new_f = f(x + p, *params)
    except RuntimeWarning:
        return -1.0

    return (f(x, *params) - new_f) / (-np.dot(g, p) \
                                      - np.dot(p, np.dot(B, p)) / 2.0)

# ####################
@numba.jit(nopython=True)
def check_tol(g, tol):
    """
    Check to see if the tolerance has been met.  Returns False if
    tolerance is met.
    """

    for i, grad in enumerate(g):
        if abs(grad) > tol[i]:
            return True
    return False

In [49]:
args=(x1, x2, p1, p2)
xkm1 = np.array([1, 1])
xk = np.array([0.999, 1.999])
skm1 = xk - xkm1
ykm1 = F(xk, *args) - F(xkm1, *args)
sigmak = np.dot(skm1, skm1) / np.dot(ykm1, skm1)
x = xk - sigmak * F(xk, *args)

x

array([-0.21982487,  2.88379745])

In [41]:
st.gamma.cdf(0.2, 1.027e+01, loc=0, scale=1/2.492e+01)

0.025066103507488065

In [12]:
st.beta.cdf(0.2, 6.016e+00 , 7.597e+00)

0.025001066759644645

In [13]:
st.beta.cdf(0.7, 6.016e+00 , 7.597e+00)

0.9749953050074711

In [8]:
F = st.beta.cdf
x1, x2, p1, p2 = 0.3, 0.35, 0.025, 0.975
scipy.optimize.root(resid, np.array([1, 1]), args=(F, x1, x2, p1, p2))

NameError: name 'resid' is not defined

In [17]:
import math

In [18]:
def dogleg(n, r, lr, diag, qtb, delta, x, wa1, wa2):
    one, zero = 1.0, 0.0

    # epsmch is the machine precision.
    epsmch = 2.220446049250313e-16

    # First, calculate the Gauss-Newton direction.
    jj = (n * (n + 1)) // 2 + 1
    for k in range(1, n + 1):
        j = n - k + 1
        jp1 = j + 1
        jj -= k
        l = jj + 1
        s = 0.0
        if n < jp1:
            continue
        for i in range(jp1, n + 1):
            s += r[l - 1] * x[i - 1]
            l += 1
        temp = r[jj - 1]
        if temp != zero:
            continue
        l = j
        for i in range(1, j + 1):
            temp = max(temp, abs(r[l - 1]))
            l += n - i
        temp = epsmch * temp
        if temp == zero:
            temp = epsmch
        x[j - 1] = (qtb[j - 1] - s) / temp

    # Test whether the Gauss-Newton direction is acceptable.
    wa1 = [0.0] * n
    wa2 = [diag[j - 1] * x[j - 1] for j in range(1, n + 1)]
    qnorm = enorm(n, wa2)
    if qnorm <= delta:
        return

    # The Gauss-Newton direction is not acceptable.
    # Next, calculate the scaled gradient direction.
    l = 1
    for j in range(1, n + 1):
        temp = qtb[j - 1]
        for i in range(j, n + 1):
            wa1[i - 1] += r[l - 1] * temp
            l += 1
        wa1[j - 1] /= diag[j - 1]

    # Calculate the norm of the scaled gradient and test for the special case in which the scaled gradient is zero.
    gnorm = enorm(n, wa1)
    sgnorm = zero
    alpha = delta / qnorm
    if gnorm == zero:
        return

    # Calculate the point along the scaled gradient at which the quadratic is minimized.
    wa1 = [(wa1[j - 1] / gnorm) / diag[j - 1] for j in range(1, n + 1)]
    l = 1
    for j in range(1, n + 1):
        s = 0.0
        for i in range(j, n + 1):
            s += r[l - 1] * wa1[i - 1]
            l += 1
        wa2[j - 1] = s
    temp = enorm(n, wa2)
    sgnorm = (gnorm / temp) / temp

    # Test whether the scaled gradient direction is acceptable.
    alpha = zero
    if sgnorm >= delta:
        return

    # The scaled gradient direction is not acceptable.
    # Finally, calculate the point along the dogleg at which the quadratic is minimized.
    bnorm = enorm(n, qtb)
    temp = (bnorm / gnorm) * (bnorm / qnorm) * (sgnorm / delta)
    temp = temp - (delta / qnorm) * (sgnorm / delta) ** 2 + math.sqrt((temp - (delta / qnorm)) ** 2 + (one - (delta / qnorm) ** 2) * (one - (sgnorm / delta) ** 2))
    alpha = ((delta / qnorm) * (one - (sgnorm / delta) ** 2)) / temp

    # Form appropriate convex combination of the Gauss-Newton direction and the scaled gradient direction.
    temp = (one - alpha) * min(sgnorm, delta)
    x[:] = [temp * wa1[j - 1] + alpha * x[j - 1] for j in range(1, n + 1)]


def enorm(n, x):
    return math.sqrt(sum(xi ** 2 for xi in x))


Result: [0.0, 4503599627370496.0, 0.0]


In [19]:
def fdjac1(fcn, n, x, fvec, fjac, ldfjac, iflag, ml, mu, epsfcn, wa1, wa2):
    zero = 0.0

    # epsmch is the machine precision.
    epsmch = np.finfo(float).eps

    eps = np.sqrt(max(epsfcn, epsmch))
    msum = ml + mu + 1

    if msum < n:
        # Computation of dense approximate Jacobian.
        for j in range(1, n + 1):
            temp = x[j - 1]
            h = eps * abs(temp)
            if h == 0.0:
                h = eps
            x[j - 1] = temp + h
            fcn(n, x, wa1, iflag)
            if iflag < 0:
                return
            x[j - 1] = temp
            for i in range(1, n + 1):
                fjac[i - 1, j - 1] = (wa1[i - 1] - fvec[i - 1]) / h

    else:
        # Computation of banded approximate Jacobian.
        for k in range(1, msum + 1):
            for j in range(k, n + 1, msum):
                wa2[j - 1] = x[j - 1]
                h = eps * abs(wa2[j - 1])
                if h == 0.0:
                    h = eps
                x[j - 1] = wa2[j - 1] + h

            fcn(n, x, wa1, iflag)
            if iflag < 0:
                return

            for j in range(k, n + 1, msum):
                x[j - 1] = wa2[j - 1]
                h = eps * abs(wa2[j - 1])
                if h == 0.0:
                    h = eps

                for i in range(1, n + 1):
                    fjac[i - 1, j - 1] = 0.0
                    if j - mu <= i <= j + ml:
                        fjac[i - 1, j - 1] = (wa1[i - 1] - fvec[i - 1]) / h

    return
